<a href="https://colab.research.google.com/github/passivebook/public/blob/main/CSV_to_Affiliate_Links.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://drive.google.com/drive/folders/1T5JJmlDwdaEkAAbpH2VOn-TCeAuW8OKl

In [ ]:
import os
import pandas as pd
import re
from bs4 import BeautifulSoup
from bs4 import element as bs4_element
from google.colab import drive
import csv
drive.mount("/content/gdrive")

Mounted at /content/gdrive


#### Put the path to the google drive folder after `/content/gdrive/My Drive/`



In [ ]:
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Fix WordPress Links")

#### Upload the post.txt file to Google Drive 
1. Create a `post.txt` file and paste the WordPress Post Code in it. 
2. Upload the post.txt file to path setup above. 

### Check file post.txt exists in Drive by running the code below

In [ ]:
! ls

 affiliate_links.csv		     'Fix WordPress Links.ipynb'   input.txt
'Copy of Fix WordPress Links.ipynb'   index_hyperlinked.html	   output.txt


## Affiliate Link

In [ ]:
html_doc = """
<!-- wp:paragraph -->
<p>This is a existing link <a class="test" href="https://yahoo.com/">Yahoo</a> Text</p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p>This is another Google Text</p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p>This is another lowercase bing Text</p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p>This is another multi word Active Campaign Text</p>
<!-- /wp:paragraph -->
"""

soup = BeautifulSoup(html_doc, 'html.parser')

# read the CSV file with anchor text and hyperlinks
with open('file.csv', 'r') as csv_file:
  reader = csv.reader(csv_file)
  hyperlinks = dict(reader)

In [ ]:
# from bs4 import element as bs4_element
be_navStr = bs4_element.NavigableString

hList = [
    (anchor_text.strip(), hyperlink.strip()) for
    anchor_text, hyperlink in hyperlinks.items()
    if anchor_text.strip() and hyperlink.strip() # no blanks
]

In [ ]:
html_doc = """
<!-- wp:paragraph -->
<p>This is a existing link <a class="test" href="https://yahoo.com/">Yahoo</a> Text</p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p>This is another Google Text</p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p>This is another lowercase bing Text</p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p>This is another multi word Active Campaign Text</p>
<!-- /wp:paragraph -->
"""

soup = BeautifulSoup(html_doc, 'html.parser')

# read the CSV file with anchor text and hyperlinks
with open('affiliate_links.csv', 'r') as csv_file:
  reader = csv.reader(csv_file)
  hyperlinks = dict(reader)

# from bs4 import element as bs4_element
be_navStr = bs4_element.NavigableString

hList = [
    (anchor_text.strip(), hyperlink.strip()) for
    anchor_text, hyperlink in hyperlinks.items()
    if anchor_text.strip() and hyperlink.strip() # no blanks
]


for txt, link in hList:
    navStrs = [
        d for d in soup.descendants if type(d) == be_navStr 
        and f' {txt.lower()} ' in f' {d.string().strip().lower()} '
    ]
    for ns in navStrs: 
        tLen, remStr = len(txt), f' {ns.get_text().strip()} '
        if remStr[1:-1].lower() == txt.lower():
            # to skip if it's already a hyperlink
            if ns.parent.name == 'a': 
                # ns.parent['href'] = link # if you want to replace/update link
                continue 

        while f' {txt.lower()} ' in remStr.lower():
            sInd = remStr.lower().find(f' {txt.lower()} ') + 1

            hlTag = soup.new_tag('a', href=link)
            hlTag.append(remStr[sInd:sInd + tLen])

            newCont = [remStr[:sInd].lstrip(), hlTag, ' ']
            for addn in newCont: ns.insert_before(addn) 

TypeError: ignored